<a href="https://colab.research.google.com/github/singwang-cn/Neural-Network/blob/master/ResNet34_Implementation_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.nn import functional as F
import torchvision
import torchvision.transforms as T
import torchvision.models as M
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class ResBlock(nn.Module):
  def __init__(self, inp_channels, num_channels, use_1x1conv=False, strides=1) -> None:
    super().__init__()

    self.conv1 = nn.Conv2d (in_channels=inp_channels, out_channels=num_channels, kernel_size=3, stride=strides, padding=1)
    self.conv2 = nn.Conv2d (in_channels=num_channels, out_channels=num_channels, kernel_size=3, stride=1, padding=1)
    self.conv3 = None
    if use_1x1conv:
      self.conv3 = nn.Conv2d (in_channels=inp_channels, out_channels=num_channels, kernel_size=1, stride=strides)

    self.bn1 = nn.BatchNorm2d(num_channels)
    self.bn2 = nn.BatchNorm2d(num_channels)
    self.relu = nn.ReLU(inplace=True)

  def forward(self, X):
    Y = self.conv1(X)
    Y = self.bn1(Y)
    Y = F.relu(Y)

    Y = self.conv2(Y)
    Y = self.bn2(Y)

    if self.conv3:
      X = self.conv3(X)
    Y += X
    Y = F.relu(Y)
    return Y


In [3]:
def generate_blocks(input_channels, num_channels, num_blocks, first_block=False):
  blocks = []
  for i in range(num_blocks):
    if i == 0 and not first_block:
      blocks.append(ResBlock(input_channels, num_channels, use_1x1conv=True, strides=2))
    else:
      blocks.append(ResBlock(num_channels, num_channels))
  return blocks


In [4]:
def build_34layer_resnet(num_classes=1000):
  block1 = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
  )
  block2 = nn.Sequential(*generate_blocks(64, 64, 3, first_block=True))
  block3 = nn.Sequential(*generate_blocks(64, 128, 4))
  block4 = nn.Sequential(*generate_blocks(128, 256, 6))
  block5 = nn.Sequential(*generate_blocks(256, 512, 3))
  fc = nn.Sequential(
      nn.AdaptiveAvgPool2d((1, 1)),
      nn.Flatten(start_dim=1, end_dim=-1),
      nn.Linear(512, num_classes)
  )

  return nn.Sequential(block1, block2, block3, block4, block5, fc)


In [5]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    """https://github.com/pytorch/examples/blob/main/imagenet/main.py"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(1.0 / batch_size))
        return res

In [12]:
def start_train(model, loader_tr, loader_va, show_epoch=1, num_epochs=10):
  dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
  model.type(dtype)
  loss_func = nn.CrossEntropyLoss().type(dtype)
  optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, min_lr=1e-5, verbose=True)
  cost = []

  start_time = time.time()

  for epoch in range(num_epochs):
    for x, y in loader_tr:
      y_pred = model(x.type(dtype)).cpu()
      loss = loss_func(y_pred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      cost.append(loss.detach().numpy())

    if (epoch+1) % show_epoch == 0:
      y_outp = []
      target = []
      for x, y in loader_va:
        y_outp.append(model(x.type(dtype)).cpu().detach())
        target.append(y)
      y_outp = torch.cat(y_outp)
      target = torch.cat(target)
      accu_top1 = accuracy(y_outp, target, topk=(1,))[0].numpy()[0]
      accu_top5 = accuracy(y_outp, target, topk=(5,))[0].numpy()[0]
      loss = loss_func(y_outp, target)
      time_current = time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))
      scheduler.step(accu_top1)
    
      print(f"epoch:{epoch+1}, loss@val:{loss:.3f} acurracy@top1:{accu_top1:.3f}, acurracy@top5:{accu_top5:.3f}, time:{time_current}")


In [13]:
transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar10_tr = dset.CIFAR10("./dataset/cifar10", train=True, download=True, transform=transform_train)
cifar10_te = dset.CIFAR10("./dataset/cifar10", train=False, download=True, transform=transform_test)

NUM_TRAIN = 50000
NUM_VALID = 10000

batch_size = 256

loader_tr = DataLoader(cifar10_tr, batch_size=batch_size, shuffle=True)
loader_va = DataLoader(cifar10_te, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
model = build_34layer_resnet(num_classes=10)
#model = M.resnet34()
#model.fc = nn.Linear(in_features=512, out_features=100, bias=True)

In [15]:
print(model)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [17]:
start_train(model, loader_tr, loader_va, show_epoch=5, num_epochs=50)

epoch:5, loss@val:0.576 acurracy@top1:0.803, acurracy@top5:0.990, time:00:02:28
epoch:10, loss@val:0.554 acurracy@top1:0.811, acurracy@top5:0.988, time:00:04:55
epoch:15, loss@val:0.510 acurracy@top1:0.834, acurracy@top5:0.990, time:00:07:23
epoch:20, loss@val:0.498 acurracy@top1:0.836, acurracy@top5:0.991, time:00:09:51
epoch:25, loss@val:0.489 acurracy@top1:0.838, acurracy@top5:0.990, time:00:12:18
epoch:30, loss@val:0.480 acurracy@top1:0.849, acurracy@top5:0.991, time:00:14:46
epoch:35, loss@val:0.458 acurracy@top1:0.858, acurracy@top5:0.992, time:00:17:13
epoch:40, loss@val:0.484 acurracy@top1:0.848, acurracy@top5:0.991, time:00:19:41
epoch:45, loss@val:0.464 acurracy@top1:0.862, acurracy@top5:0.993, time:00:22:08
epoch:50, loss@val:0.430 acurracy@top1:0.868, acurracy@top5:0.993, time:00:24:35


In [ ]:
#https://pystyle.info/pytorch-resnet/
#https://github.com/kuangliu/pytorch-cifar/blob/master/main.py